# Generic Mapping Toolbox (GMT) maps with Julia
Switching to Julia with concerns how to plot your data? Make your maps look great again with GMT! This notebook will show some of the amazing GMT+Julia features. 

### Installation
* [GMT](http://gmt.soest.hawaii.edu) and set to PATH (e.g., `"e:\programs\gmt5\bin"`)
* [Ghostscript](https://ghostscript.com) and set to PATH (e.g., `"e:\Program Files\gs\gs9.23\bin"`)
* run `Pkg.add("GMT")` to install directly via REPL 

In case you need to see some serious **docs** before the liftoff:
* User guide for [GMT.jl](https://genericmappingtools.github.io/GMT.jl/latest/) and [GMT](http://gmt.soest.hawaii.edu/doc/5.4.3/index.html)


In [28]:
using GMT, DataFrames

### Basic GMT settings
* Following examples are related to calls from Julia
    * _Julia example:_ `gmt("set PS_MEDIA A4")` _vs command prompt:_ `gmtset PS_MEDIA A4`
* Format of the plot paper: 
    * to ISO A4: `set PS_MEDIA A4`
    * to custom size, e.g. 10x6 cm: `Custom_10cx6c`
    * the `c` means centimeters (for inches, use `i`)
* Format fonts:
    * annotation font (size,type): `set FONT_ANNOT_PRIMARY 9p,Helvetica`
    * labels: `set FONT_LABEL 7p`
* Set frame (appearance, not spacing etc.)
    * frame width (how thick will it be=0.08 cm): `set MAP_FRAME_WIDTH 0.08c`
    * frame thicks length: `tset MAP_TICK_LENGTH 0.08c`

In [29]:
# Set the format of the plot paper
gmt("set PS_MEDIA A4")
# set fonts 
gmt("set FONT_ANNOT_PRIMARY 8p,Helvetica")
gmt("set FONT_LABEL 6p")
# set frame
gmt("set MAP_TICK_LENGTH 0.07c")
gmt("set MAP_FRAME_WIDTH  0.07c")

### Coastlines/land
* Can be applied calling `coast()` function in Julia (or via command prompt using `pscoast`)
* First, set [projection](http://gmt.soest.hawaii.edu/doc/latest/gmt.html#j-full), e.g.: 
    * Mercator with central meridian at 0 degrees and map width of 9 cm: `proj="M0/9c"` (`-JM09c`)
    * Cylindrical Equidistant projection with 0 deg. and map with of 9.5 cm: `proj="Q0/9.5c"` (`-JQ0/9.5c`)
* Set the plotting [region](http://gmt.soest.hawaii.edu/doc/latest/gmt.html#r-full): 
    * to full globe: `region=[-180 180 -90 90]` (`-R-180/180/-90/90`)
* Set boundary/[frame](http://gmt.soest.hawaii.edu/doc/latest/gmt.html#b-full): 
    * annotation every 45 deg. longitude, 30 deg. latitude while showing anotation only on South+East (west+north only frame=capital letters): `frame="45g/30gSEnw"` (`-B45g45/30g30SEnw`)
    * annotation longitude 40 deg., thicks every 20 deg. (without plotting lines=f) + 30 deg. latitude and thicks every 15 deg., and on all frame sites: `frame="40f20/30f15SENW"` (`-B40f20/30f15SENW`)
* Plot [land](http://gmt.soest.hawaii.edu/doc/latest/coast.html#g), [ocean](http://gmt.soest.hawaii.edu/doc/latest/coast.html#s), and [rivers](http://gmt.soest.hawaii.edu/doc/latest/coast.html#g)
    * fill land area with gray: `land="gray"` (`-Ggray`)
    * plot features with area > 1e+4 km^2: `area=10000` (`-A10000`)
    * permanent major rivers, width=0.2, in blue: `rivers="1/0.2p,blue"` (`-I1/0.2p,blue`)
    * ocean in blue: `water="skyblue1"` (`Sskyblue1`)
* Plot [borders](http://gmt.soest.hawaii.edu/doc/latest/coast.html#n)
    * all national/political boarders in dark gray and with 0.1p: `borders="1/0.1p,gray10"` (`-N1/0.1p,gray90`)
* Set data [resolution](http://gmt.soest.hawaii.edu/doc/latest/coast.html#d)
    * low resolution of the data (not output picture!): `res="l"` (`-Dl`)
* To show the results use `show=true` (will open the temp file)

In [30]:
coast(proj="Q12c", # cylindrical equidistant projection, 12cm 
    region=[-180 180 -80 85], # plot almost the whole globe
    frame="60f30/40f20SEnw", # set thicks every 20/15 (lon/lat), annotation 40/30 (lon/lat) degrees
    land="khaki",area=10000, # plot land > 10000 km^2 in gray
    water="skyblue1", # ocena in sky/light blue
    rivers="2/,skyblue1", # plot "additional major" rivers with default width in light blue
    borders="1/0.1p,gray25", # plot political borders in 25% gray (100=black)
    res="l") # low data resolution

### Plot xy data
* Function `plot` is equivalent of [psxy](http://gmt.soest.hawaii.edu/doc/5.4.3/psxy.html) in GMT
* Can use the same projection settings as described in section **Coastlines/land**
* First two arguments in plot are X (longitude) and Y (latitude)
    * add data xy (x=10,y=30) data to existing figure: `plot!([10.],[30.],marker="plus")`
    * in GMT the X,Y values should be in ASCII file: `psxy XY.txt -S+ -R -J`
* Specify marker
    * red, full circle with size of 2p: `plot!([1],[2],marker="circle",markeredgecolor="red", markerfacecolor="red",size="2p")`
    * in GMT: `psxy XY.txt -Sc -W2p,red -Gred`

In [31]:
## prepare data that should be plotted
sites = DataFrame(
        name=["Cibinong", "Cantley", "Djougou", "Tigo", "Pecny", "Sutherland"],
        longitude=[106.85, -75.8071, 1.6056, -73.0256, 14.7856, 20.8109],
        latitude =[-6.4903, 45.585, 9.7424, -36.8438, 49.9138, -32.3814],
        climate  =["A", "D", "A", "C", "D", "B"]);

## plot data 
plot!(sites[:longitude], sites[:latitude], 
    marker="circle",markeredgecolor="black", markerfacecolor="red",
    size=0.08)

### Plot text
* Function `text` is equivalent to [pstext](http://gmt.soest.hawaii.edu/doc/5.4.3/pstext.html) in GMT
* Can use the same projection settings as described in section Coastlines/land
* Need to prepare input for `gmt` function
    * string vector with "X Y text": `textin = ["10 20 first", "10 30 second"];`
    * in GMT the X,Y values and text in ASCII file: `pstext XYtext.txt -R -J -F+f6p,Helvetica -Gwhite`
* Set font size to Helvetica 6p, red: `text_attrib="+f6p,Helvetica,red"` (`-F+f6p,Helvetica,red"`)
* Set white background: `fill=white` (`-Gwhite`)
* Set dX and dY offset (shift in longitude and latitude) to 0.2 and zero: `D="0.2/0"` (`-D0.2/0`)

In [32]:
## prepare text vector 
textin = Vector{String}(0);
for (i,v) in enumerate(sites[:name])
    push!(textin,@sprintf("%.4f %.4f %s",sites[:longitude][i],sites[:latitude][i],v))
end

## plot text and show result as png
text!(F ="+f7p,Helvetica,black",D="0/0.18",
    fmt="png",show=true, # plot to PNG file and show result => will open created figure
    textin) # input with coordinates and text 

### Resulting map 
<img src='images/gmt_coast_text.png'>

In [33]:
# reset GMT config
gmt("destroy")